## **Solutions parallèles pour la résolution des systèmes d'équations linéaires par la méthode d'élimination Gauss-Jordan**

-**Réalisé par :** 
> - *Lamdani Wilem (2CS SIQ3)*
> - *Belkessa Linda (2CS SIQ3)*

- **Encadré par :**
> - *Mme. Haichour Salima*



### **Présentation de l'algorithme d'élimination Gauss-Jordan :**
> L'élimination Gauss-Jordan vise à transformer un système d'équations linéaires en une matrice triangulaire supérieure afin de résoudre les inconnues et d'en déduire une solution. Une colonne pivot est utilisée pour réduire les lignes qui la précèdent ; puis après la transformation, la back-substitution est appliquée.

> ![image1](https://i.ibb.co/XxfnZVH/gelim-cuda-1.png)

> **Exemple :**

> (1) Pour effectuer une élimination gaussienne en commençant par le système d'équations :
![image2](https://media.cheggcdn.com/study/e2c/e2c66011-05e7-4974-baf9-29eb251a44aa/DC-1723V1.png)

> (2) Composer la matrice augmentée 

>![image3](https://media.cheggcdn.com/study/1f2/1f287abc-807e-4412-86f7-9db40d587115/DC-1723V2.png)

> (3) Mettre la matrice augmentée sous forme de matrice triangulaire supérieure 

> ![image3](https://media.cheggcdn.com/study/cae/cae5db46-d3b6-4909-8764-185a6398c5c3/DC-1723V3.png)

> (4) Résooudre l'équation de la ième ligne pour , puis remettez-la dans l'équation de la (k-1)ère ligne pour
obtenir une solution pour Xk-1, etc., selon la formule

>![Formule](https://i.ibb.co/FYnhh9W/image-2022-02-03-230208.png)

>![image4](https://media.cheggcdn.com/study/0ac/0ac1e07c-7f57-4c91-9d45-f3db808dad1c/DC-1723V4.png)

### **Implémentation séquentielle :**
- **Principe :**    
***
```C
Loop 1 : for (norm = 0; norm < N - 1; norm++) {
            Loop 2 : for (row = norm + 1; row < N; row++) {
                                     multiplier = A[row][norm] / A[norm][norm];
                      Loop 3 : for (col = norm; col < N; col++) {
                                            A[row][col] -= A[norm][col] * multiplier;
                                     }
                                     B[row] -= B[norm] * multiplier;
                           }
                }

```
***



### **Méthodologie de parallèlisation avec openMP :**
Pour pouvoir bien utiliser openMP, il faut dans un premier temps répondre aux questions suivantes :    
1- Quelles sont les boucles parallèlisable ? 
> - Les boucles dont le nombre d'itérations est connu dès le départ et qui ne change pas
> - Les boucles où chaque itération est indépendante des autres
> - Les boucles qui ne contiennet pas une dépendence de données (avec les boucles externes en général)

Observons d'abord comment s'effectue l'élimination de Gauss-Jordan visuellement, 4 scénarios sont possibles:      
> ![gauss-jordan viz](https://i.ibb.co/3TbRjb3/Gaussian-Elimination-1.png)

- La boucle i est représentée par la ligne et la colonne jaunes. Les entrées dans la ligne et la colonne jaunes sont utilisées pour mettre à jour la sous-matrice verte avant de passer à la ligne/colonne i+1, ce qui signifie que les valeurs des entrées dans la (i+1)ère zone jaune dépendent des opérations effectuées sur eux aux valeurs précédentes de i. Par conséquent, nous ne pouvons pas utiliser OpenMP pour paralléliser cette boucle en raison de la dépendance des données.

- La boucle j a un nombre d'itérations qui varie avec i, mais nous connaissons le nombre d'itérations à chaque fois que nous sommes sur le point d'entrer dans la boucle. Aucune des itérations ultérieures ne dépend des précédentes et les itérations peuvent être calculées dans n'importe quel ordre ! La boucle j est donc parallélisable.

- La boucle k, comme la boucle j, a un nombre d'itérations qui varie mais qui est calculable pour chaque i. Aucune des itérations ultérieures ne dépend des précédentes, et elles peuvent toutes être calculées dans n'importe quel ordre. Par conséquent, la boucle k est également parallélisable.

> __*Conclusion :*__ Il est préférable de sélectionner la boucle externe (j), car nous aurons alors plus de parallélisme ininterrompu et moins de fork et de join.




### **Implémentation du code avec openMP:**     
> 1. La boucle externe (Loop 1) ne peut pas etre parallèlisée car elle contient une variable liée au controle de la boucle interne (Loop 2)

> 2. La boucle interne (Loop 2) nne contient aucune dépendence de donnée d'où elle est parallèlisable

> 3. La matrice A, le vecteur B et l'index de la boucle externe 'norm' et N seront partagées (shared).

> 4. Idée globale : Durant la première itération de la boucle externe (Loop 1), on initialise toutes les entrées dans column[0] en commençant de row[1]. 
Dand la boucle interne (Loop 2) chaque thread s'occupe d'additionner une ligne un multiple (par un scalaire) d'une autre ligne à fin d'avoir 0. à la sortie de la boucle interne, la boucle externe passe à la prochaine colonne, il s'agit ici de l'échelonnement par colonnes. Puis, on effectue une opération de backsubstitution pour résoudre l'équation.

> #### **Remarque importante:** 
> Google Colab n'offre dans cette version gratuite que 2 processeurs chacun avec 1 core, ce qui n'est pas suffisant pour comparer l'exécution séquentielle et parallèle, il est souhaitable d'exécuter le code openMP avec des ressources locaux.

Pour dérouler le code openMP/Séqueniel localement, on peut utiliser un environnement d'exécution locale pour exploiter les coeurs CPU de la machine, pour celà :
> - Dans le dossier drice, télécharger 'sequentiel.c' et 'openMP.c' et 'notebook_openMP.ipynb'.
- Installer jupyter notebook localement `pip install notebook`
- Lancer un notebook local `jupyter notebook`
- Téléverser les fichiers téléchargés dans le répértoire local et lancer l'exécution.
- Ou utiliser un IDE (avec compilateur gcc)

In [1]:
!gcc sequentiel.c -o sequentiel

In [4]:
!sequentiel 256 2


Dimension de la matrice N = 256. Seed (germe) = 2 .

Initialisation...
Execution sequentielle ... .

Temps d'exÃ©cution SÃ©quentiel = 12.965 ms.


In [5]:
!sequentiel 512 2


Dimension de la matrice N = 512. Seed (germe) = 2 .

Initialisation...
Execution sequentielle ... .

Temps d'exÃ©cution SÃ©quentiel = 92.065 ms.


In [6]:
!sequentiel 1024 2


Dimension de la matrice N = 1024. Seed (germe) = 2 .

Initialisation...
Execution sequentielle ... .

Temps d'exÃ©cution SÃ©quentiel = 754.473 ms.


In [7]:
!sequentiel 2048 2


Dimension de la matrice N = 2048. Seed (germe) = 2 .

Initialisation...
Execution sequentielle ... .

Temps d'exÃ©cution SÃ©quentiel = 6442.76 ms.


In [10]:
##2 cores
!gcc -fopenmp openMP.c -o openmp

In [12]:
!openmp 256 2


Dimension de la matrice N = 256. Seed (germe) = 2 .

Initialisation...
Execution parallele avec openMP.

Temps d'exÃ©cution avec openMP = 15.634 ms.


In [13]:
!openmp 512 2


Dimension de la matrice N = 512. Seed (germe) = 2 .

Initialisation...
Execution parallele avec openMP.

Temps d'exÃ©cution avec openMP = 71.215 ms.


In [14]:
!openmp 1024 2


Dimension de la matrice N = 1024. Seed (germe) = 2 .

Initialisation...
Execution parallele avec openMP.

Temps d'exÃ©cution avec openMP = 451.438 ms.


In [15]:
!openmp 2048 2


Dimension de la matrice N = 2048. Seed (germe) = 2 .

Initialisation...
Execution parallele avec openMP.

Temps d'exÃ©cution avec openMP = 3430.68 ms.


In [23]:
## 4 cores
!gcc -fopenmp openMP.c -o openmp

In [24]:
!openmp 256 2


Dimension de la matrice N = 256. Seed (germe) = 2 .

Initialisation...
Execution parallele avec openMP.

Temps d'exÃ©cution avec openMP = 26.537 ms.


In [25]:
!openmp 512 2


Dimension de la matrice N = 512. Seed (germe) = 2 .

Initialisation...
Execution parallele avec openMP.

Temps d'exÃ©cution avec openMP = 41.481 ms.


In [26]:
!openmp 1024 2


Dimension de la matrice N = 1024. Seed (germe) = 2 .

Initialisation...
Execution parallele avec openMP.

Temps d'exÃ©cution avec openMP = 340.611 ms.


In [27]:
!openmp 2048 2


Dimension de la matrice N = 2048. Seed (germe) = 2 .

Initialisation...
Execution parallele avec openMP.

Temps d'exÃ©cution avec openMP = 2426.51 ms.
